In [1]:
from newsapi import NewsApiClient

In [2]:
import requests
import json

In [3]:
from string import Template

In [4]:
from bs4 import *

In [5]:
from selenium import webdriver

In [6]:
import time

In [ ]:
url = ("""https://newsapi.org/v2/everything?
language=en
&from=2019-10-01
&sources=cnbc
&apiKey=749e4a9b92e046a386a4420df0de07ee""")

requests.get(url)


In [ ]:
url_template = Template("""
https://newsapi.org/v2/everything?\
language=$language\
&from=$from\
&sources=$sources\
&apiKey=$apiKey\
&pageSize=$pageSize\
&page=$page\
""")

In [ ]:
url = url_template.substitute({"language": "en", "from": "2019-10-01", "sources": "cnbc", 
                               "apiKey": "749e4a9b92e046a386a4420df0de07ee", "pageSize": 30, "page": 2})

In [ ]:
url

In [ ]:
response = requests.get(url)

In [ ]:
json_response=response.json()

In [ ]:
len(json_response["articles"])

In [ ]:
for entity in json_response:
    print(entity)

In [ ]:
news_articles=[]
for entity in json_response:
    if(entity=='articles'):
        for i in range(0,len(json_response[entity])):
            news_articles.append(json_response['articles'][i]['content'])     

In [ ]:
news_articles[0]

In [ ]:
len(news_articles)

In [ ]:
# Selenium web driver.
driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver")

In [11]:
# Function to get and store CNBC news articles (text and JSON) in respective directories.

def get_news_articles_cnbc (from_date="2019-10-01", api_key="749e4a9b92e046a386a4420df0de07ee", page_size=100, pages=3,
                            start_page=1, json_data_dir="C:/", text_data_dir="C:/",
                            chrome_driver="C:/Users/Administrator/chromedriver_win32/chromedriver"):
    # Create the URL template to query News API.
    url_template = Template("""
    https://newsapi.org/v2/everything?\
    language=$language\
    &from=$from\
    &sources=$sources\
    &apiKey=$apiKey\
    &pageSize=$pageSize\
    &page=$page\
    """)
    # Get the driver object.
    driver = webdriver.Chrome(chrome_driver)
    
    # Iterate over a number of pages.
    for page in range(start_page, pages):
        url = url_template.substitute({"language": "en", "from": from_date, "sources": "cnbc", 
                                       "apiKey": api_key, "pageSize": page_size, "page": page})
        response = requests.get(url)
        print(response.json())
        # Iterate over page size, as the number of articles retrieved is = page size.
        for article in range(1, page_size):
            # File name - Article + underscore + time.
            news_article_fname = "article_" + time.strftime("%Y%m%d_%H%M%S")
            # Article - In text and JSON formats.
            news_article_text = ""
            news_article_json = ""
            print(response.json().keys())
            # Check if articles are fetched, if not - continue.
            if ("articles" in response.json().keys()):
                # Get article URL. We receive this by parsing the News API response.
                news_url = response.json()["articles"][article]["url"]
                # Print the current status.
                print("INFO: Fetching article - " + str(article) + " of page - " + str(page) + ". URL - " + news_url)
                # Get page using the Chromium driver.
                news_article = driver.get(news_url)
                # From this page, get the inner HTML. This gets the entire page as one HTML Document.
                news_article_html = driver.find_element_by_xpath("//*").get_attribute("innerHTML")
                # Beautiful soup parsed.
                news_article_bs = BeautifulSoup(news_article_html, "lxml")
                # For CNBC, the structure is - one div with id = MainContent, and that consists of divs in class - group.
                news_article_divs = news_article_bs.find(id="MainContent").find_all("div", class_="group")
                # Iterate over the divs to fetch and concatenate paragraphs.
                for div in news_article_divs:
                    # Get all paragraphs in that div.
                    news_article_p = div.find_all("p")
                    for p in news_article_p:
                        # Concatenate the text string with the new data from the paragraph tag of the news article.
                        news_article_text = news_article_text + "\n" + p.get_text()
                # If the text is empty, don't dump to file.
                if not (news_article_text.strip()==""):
                    # Write JSON, using JSON.DUMPS function.
                    news_article_jfile = open(json_data_dir + "/" + news_article_fname + ".json", "w+", encoding="utf-8")
                    news_article_jfile.write(json.dumps(response.json()["articles"][article]))
                    news_article_jfile.close()
                    # Write Text.
                    news_article_tfile = open(text_data_dir + "/" + news_article_fname + ".txt", "w+", encoding="utf-8")
                    news_article_tfile.write(news_article_text.strip())
                    news_article_tfile.close()
    # Close Selenium Chrome driver.
    driver.close()

In [38]:
# Function to get and store CNBC news articles (text and JSON) in respective directories.

def get_news_articles_bi (from_date="2019-10-01", api_key="749e4a9b92e046a386a4420df0de07ee", page_size=20, pages=10,
                            start_page=1, json_data_dir="C:/", text_data_dir="C:/",
                            chrome_driver="C:/Users/Administrator/chromedriver_win32/chromedriver"):
    # Create the URL template to query News API.
    url_template = Template("""
    https://newsapi.org/v2/everything?\
    language=$language\
    &from=$from\
    &sources=$sources\
    &apiKey=$apiKey\
    &pageSize=$pageSize\
    &page=$page\
    """)
    # Get the driver object.
    driver = webdriver.Chrome(chrome_driver)
    
    # Iterate over a number of pages.
    for page in range(start_page, pages):
        url = url_template.substitute({"language": "en", "from": from_date, "sources": "business-insider", 
                                       "apiKey": api_key, "pageSize": page_size, "page": page})
        response = requests.get(url)
        # Iterate over page size, as the number of articles retrieved is = page size.
        for article in range(1, page_size):
            # File name - Article + underscore + time.
            news_article_fname = "article_" + time.strftime("%Y%m%d_%H%M%S")
            # Article - In text and JSON formats.
            news_article_text = ""
            news_article_json = ""
            # Check if articles are fetched, if not - continue.
            if ("articles" in response.json().keys()):
                # Get article URL. We receive this by parsing the News API response.
                news_url = response.json()["articles"][article]["url"]
                # Print the current status.
                print("INFO: Fetching article - " + str(article) + " of page - " + str(page) + ". URL - " + news_url)
                # Get page using the Chromium driver.
                news_article = driver.get(news_url)
                # From this page, get the inner HTML. This gets the entire page as one HTML Document.
                news_article_html = driver.find_element_by_xpath("//*").get_attribute("innerHTML")
                # Beautiful soup parsed.
                news_article_bs = BeautifulSoup(news_article_html, "lxml")
                # For CNBC, the structure is - one div with id = MainContent, and that consists of divs in class - group.
                news_article_divs = news_article_bs.find(id="the_bi_content")
                # If the divs is None, then continue with the loop without parsing.
                if (news_article_divs is None):
                    continue
                news_article_divs = news_article_divs.find_all("div")
                # Iterate over the divs to fetch and concatenate paragraphs.
                for div in news_article_divs:
                    # Get all paragraphs in that div.
                    news_article_p = div.find_all("p")
                    for p in news_article_p:
                        # Concatenate the text string with the new data from the paragraph tag of the news article.
                        news_article_text = news_article_text + "\n" + p.get_text()
                # If the text is empty, don't dump to file.
                if not (news_article_text.strip()==""):
                    # Write JSON, using JSON.DUMPS function.
                    news_article_jfile = open(json_data_dir + "/" + news_article_fname + ".json", "w+", encoding="utf-8")
                    news_article_jfile.write(json.dumps(response.json()["articles"][article]))
                    news_article_jfile.close()
                    # Write Text.
                    news_article_tfile = open(text_data_dir + "/" + news_article_fname + ".txt", "w+", encoding="utf-8")
                    news_article_tfile.write(news_article_text.strip())
                    news_article_tfile.close()
    # Close Selenium Chrome driver.
    driver.close()

In [ ]:
get_news_articles_cnbc(json_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/JSON",
                       text_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/Text", start_page=2)

In [37]:
get_news_articles_bi(json_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/JSON",
                       text_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/Text", start_page=2)

{'source': {'id': 'business-insider', 'name': 'Business Insider'}, 'author': 'feedback@businessinsider.com (Tyler Lauletta), Tyler Lauletta', 'title': "Nick Bosa trolled Baker Mayfield by recreating the quarterback's most iconic college celebration", 'description': 'ESPN Nick Bosa trolled Baker Mayfield while tearing through the Cleveland Browns offensive line on Monday night. After taking down Mayfield in the backfield, Bosa celebrated by pretending to wave a flag through the air and planting it in the ground. The move …', 'url': 'https://www.businessinsider.com/video-nick-bosa-trolls-baker-mayfield-flag-celebration-2019-10', 'urlToImage': 'https://image.businessinsider.com/5d9beeeee6a381404210c5f5?width=1200&format=jpeg', 'publishedAt': '2019-10-08T02:28:14Z', 'content': 'Nick Bosa and the San Francisco 49ers defense made life tough for Baker Mayfield on Monday night.\r\nThrough just one half of football, the 49ers forced three turnovers out of the Browns, with Bosa causing havoc for

{'source': {'id': 'business-insider', 'name': 'Business Insider'}, 'author': 'Kevin Webb', 'title': 'Apple is reportedly considering a subscription bundle for Apple Music and its streaming service Apple TV Plus, but the music industry is worried it will lose money on the deal (AAPL)', 'description': 'Apple is considering packaging Apple Music and the upcoming Apple TV Plus as a combined subscription, according to a new report from the Financial Times. Apple Music is currently available for $10 a month, and Apple TV Plus will launch on November 1 for $5 pe…', 'url': 'https://www.businessinsider.com/apple-is-considering-music-and-tv-plus-subscription-bundle-report-2019-10', 'urlToImage': 'https://image.businessinsider.com/5d9bac1d880f263d7b47c003?width=1200&format=jpeg', 'publishedAt': '2019-10-07T21:45:27Z', 'content': "Apple TV Plus hasn't launched yet, but Apple has already considered packaging its on-demand video service with its streaming music service, Apple Music.\r\nA new report 

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
#Text Format
TRITTextResponse = TRITResponse.text
#print(TRITTextResponse)

#Json Format
TRITJsonResponse = json.loads(TRITTextResponse)
print(TRITJsonResponse)

for entity in TRITJsonResponse:
    for info in TRITJsonResponse[entity]:
        topics={}
        geography={}
        organizations={}
        persons={}
        industries={}
        currencies={}
        market_index={}
        
        #Topics
        if(info=='_typeGroup' and str(TRITJsonResponse[entity][info])=='topics'):
            for companyinfo in (TRITJsonResponse[entity]):
                if(companyinfo in ['_type','name','confidencelevel','score','relevance','permid']):
                    topics[companyinfo]=str(TRITJsonResponse[entity][companyinfo])
        if topics:
            topics_list.append(topics)

        
        #Geographical Locations
        if((info=='_type' and str(TRITJsonResponse[entity][info]) in ['Country','City','Continent','Region','ProvinceOrState'])):
            for companyinfo in (TRITJsonResponse[entity]):
                if(companyinfo in ['_type','name','confidencelevel','score','relevance','permid']):
                    geography[companyinfo]=str(TRITJsonResponse[entity][companyinfo])
                if(companyinfo=='resolutions'):
                    if('permid' in TRITJsonResponse[entity][companyinfo][0]):  
                        geography['permid']=str(TRITJsonResponse[entity][companyinfo][0]['permid'])
                    if('id' in TRITJsonResponse[entity][companyinfo][0]):  
                        geography['id']=str(TRITJsonResponse[entity][companyinfo][0]['id'])
                    if('score' in TRITJsonResponse[entity][companyinfo][0]):  
                        geography['score']=str(TRITJsonResponse[entity][companyinfo][0]['score'])
                    if('ticker' in TRITJsonResponse[entity][companyinfo][0]):
                        geography['ticker']=str(TRITJsonResponse[entity][companyinfo][0]['ticker'])
        if geography:
            geographical_list.append(geography)
